# Notebook 1/3 - Data Preparation

### General objective of the project
- Predict daily solar energy on next day for the city of Kraainem, in Belgium

### Notebook objective
- Download and clean raw data from solcast.com
- Prepare data a format suitable for supervised ML models (instead of TS models)
- Perform feature engineering to augment data

### Import modules

In [1]:
# import modules
import pandas as pd
import numpy as np

### Load and inspect data

In [2]:
# import raw dataset
load_path = '../data/raw/'
load_name = '50.861781_4.457452_Solcast_PT60M.csv'

df = pd.read_csv(load_path + load_name)

In [3]:
# because of the download limit from Solcast.com, all the available features were included to avoid potential gaps
df.columns

Index(['PeriodEnd', 'PeriodStart', 'Period', 'AirTemp', 'Azimuth',
       'CloudOpacity', 'DewpointTemp', 'Dhi', 'Dni', 'Ebh', 'Ghi',
       'GtiFixedTilt', 'GtiTracking', 'PrecipitableWater', 'RelativeHumidity',
       'SnowWater', 'SurfacePressure', 'WindDirection10m', 'WindSpeed10m',
       'Zenith', 'AlbedoDaily'],
      dtype='object')

In [4]:
# create a working dataframe where, a priori, unwanted features are dropped
sol = df.copy()
sol.drop(
    labels=[
        'Period',
        'Azimuth',
        'GtiFixedTilt',
        'GtiTracking',
        'DewpointTemp',
        'Ebh',
        'SnowWater',
        'Zenith',
        'AlbedoDaily'
    ],
    axis=1,
    inplace=True
)

In [5]:
# update features names for ease of use
cleaned_col_names_dict = {
    'PeriodEnd': 'End',
    'PeriodStart': 'Start',
    'AirTemp': 'Temperature',
    'WindDirection10m': 'WindDirection',
    'WindSpeed10m': 'WindSpeed'
}

sol.rename(columns=cleaned_col_names_dict, inplace=True)

In [6]:
# check dataframe size and content
sol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25989 entries, 0 to 25988
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   End                25989 non-null  object 
 1   Start              25989 non-null  object 
 2   Temperature        25989 non-null  float64
 3   CloudOpacity       25989 non-null  float64
 4   Dhi                25989 non-null  int64  
 5   Dni                25989 non-null  int64  
 6   Ghi                25989 non-null  int64  
 7   PrecipitableWater  25989 non-null  float64
 8   RelativeHumidity   25989 non-null  float64
 9   SurfacePressure    25989 non-null  float64
 10  WindDirection      25989 non-null  int64  
 11  WindSpeed          25989 non-null  float64
dtypes: float64(6), int64(4), object(2)
memory usage: 2.4+ MB


In [7]:
# check for duplicates
sol.duplicated().sum()

0

In [8]:
# double-check for null values even though none are expected
sol.isnull().sum()

End                  0
Start                0
Temperature          0
CloudOpacity         0
Dhi                  0
Dni                  0
Ghi                  0
PrecipitableWater    0
RelativeHumidity     0
SurfacePressure      0
WindDirection        0
WindSpeed            0
dtype: int64

In [9]:
# convert data to desired/proper format
sol.End = pd.to_datetime(sol.End, utc=True)
sol.Start = pd.to_datetime(sol.Start, utc=True)
sol.WindDirection = sol.WindDirection.astype(float)

In [10]:
# check and confirm dataframe size and content
sol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25989 entries, 0 to 25988
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   End                25989 non-null  datetime64[ns, UTC]
 1   Start              25989 non-null  datetime64[ns, UTC]
 2   Temperature        25989 non-null  float64            
 3   CloudOpacity       25989 non-null  float64            
 4   Dhi                25989 non-null  int64              
 5   Dni                25989 non-null  int64              
 6   Ghi                25989 non-null  int64              
 7   PrecipitableWater  25989 non-null  float64            
 8   RelativeHumidity   25989 non-null  float64            
 9   SurfacePressure    25989 non-null  float64            
 10  WindDirection      25989 non-null  float64            
 11  WindSpeed          25989 non-null  float64            
dtypes: datetime64[ns, UTC](2), float64(7), int64(3

In [11]:
# check random entries of the dataframe, look for inconsistent values despite proper formats
sol.sample(5)

,End,Start,Temperature,CloudOpacity,Dhi,Dni,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection,WindSpeed
1662,2019-03-10 08:00:00+00:00,2019-03-10 07:00:00+00:00,8.6,55.4,53,0,53,14.6,95.1,991.9,234.0,8.8
1698,2019-03-11 20:00:00+00:00,2019-03-11 19:00:00+00:00,4.0,0.2,0,0,0,6.7,63.1,1014.4,278.0,3.8
11265,2020-04-13 11:00:00+00:00,2020-04-13 10:00:00+00:00,9.6,49.2,362,6,366,12.9,57.1,1015.9,34.0,7.6
20169,2021-04-19 11:00:00+00:00,2021-04-19 10:00:00+00:00,13.0,13.2,339,398,627,15.0,54.6,1013.5,349.0,3.0
17988,2021-01-18 14:00:00+00:00,2021-01-18 13:00:00+00:00,5.7,65.7,80,0,80,10.9,84.9,1015.3,239.0,6.6


### Transform data - feature engineering

In [12]:
# create features Date from Start to prepare for aggregation of the data with this feature
sol['Date'] = pd.to_datetime(sol.Start).dt.date

In [13]:
# drop entries without radiations, such as night records for example, to focus only on day-light records in aggregation
sol.drop(
    sol[
        (sol.Dhi==0) &
        (sol.Dni==0) &
        (sol.Ghi==0)
    ].index,
    inplace=True
)

sol.shape

(13915, 13)

In [14]:
# aggregate the dataframe by Date values, i.e. day level (most granular level among Date, Year and Month feature)
# set date feature as index
sol = sol.groupby(
    'Date',
    as_index=True
).agg(
    Temp_t=pd.NamedAgg('Temperature', 'mean'),
    CloudOp_t=pd.NamedAgg('CloudOpacity', 'mean'),
    DHI_t=pd.NamedAgg('Dhi', 'mean'),
    DNI_t=pd.NamedAgg('Dni', 'mean'),
    GHI_t=pd.NamedAgg('Ghi', 'mean'),
    PrecipW_t=pd.NamedAgg('PrecipitableWater', 'mean'),
    Humid_t=pd.NamedAgg('RelativeHumidity', 'mean'),
    Press_t=pd.NamedAgg('SurfacePressure', 'mean'),
    WindDir_t=pd.NamedAgg('WindDirection', 'mean'),
    WindSpd_t=pd.NamedAgg('WindSpeed', 'mean')
)

sol.shape

(1083, 10)

In [15]:
# change index format from date to datetime
sol.index = pd.DatetimeIndex(sol.index)

In [16]:
# create list of features to further engineer
feat_eng_lst = list(sol.columns)

In [17]:
# create features Year, Month, Year_month from Date (index)
sol['Year_t'] = sol.index.year
sol['Month_t'] = sol.index.month
sol['Year_month_t'] = sol.index.to_period('M')

In [18]:
# create features min/max/mean/std of the last 7 days for each record
stats_lst = ['min', 'max', 'mean', 'std']
seven_days_rolling = sol[feat_eng_lst].rolling(7)
sol_rolling = seven_days_rolling.agg(stats_lst)

In [19]:
# create list with accurate names to rename the features of sol_rolling dataframe
feat_lst_for_df = []

for feat in feat_eng_lst:
    for stats in stats_lst:
        new_feat_name = feat + '_' + stats + '_wk'
        feat_lst_for_df.append(new_feat_name)
        
# rename columns of the rolling dataframe
sol_rolling.columns = feat_lst_for_df

In [20]:
# create a series lag features from the target feature to use ML models instead of TS
# initial data will become _t time, lag(-1) data become _t+1 time => predict t+1 using t features
one_lag_day_GHI = sol.GHI_t.shift(-1).rename('GHI_t+1')

In [21]:
# concatenate sol dataframe, sol_lagged and one_lag_day_GHI
sol_temp = pd.concat(
    [
        sol,
        sol_rolling,
        one_lag_day_GHI
    ],
    axis=1
)

In [22]:
# create list of _mean_wk features
mean_wk_feat = [
    'Temp_t_mean_wk',
    'CloudOp_t_mean_wk',
    'PrecipW_t_mean_wk',
    'Humid_t_mean_wk',
    'Press_t_mean_wk',
    'WindDir_t_mean_wk',
    'WindSpd_t_mean_wk',
    'DNI_t_mean_wk',
    'GHI_t_mean_wk',
    'DHI_t_mean_wk'    
]

# create velocity features out of mean_wk features (rate of change)
for feat in mean_wk_feat:
    sol_temp[feat + '_velocity'] = sol_temp[feat] - sol_temp[feat].shift(1)

In [23]:
sol_temp.columns

Index(['Temp_t', 'CloudOp_t', 'DHI_t', 'DNI_t', 'GHI_t', 'PrecipW_t',
       'Humid_t', 'Press_t', 'WindDir_t', 'WindSpd_t', 'Year_t', 'Month_t',
       'Year_month_t', 'Temp_t_min_wk', 'Temp_t_max_wk', 'Temp_t_mean_wk',
       'Temp_t_std_wk', 'CloudOp_t_min_wk', 'CloudOp_t_max_wk',
       'CloudOp_t_mean_wk', 'CloudOp_t_std_wk', 'DHI_t_min_wk', 'DHI_t_max_wk',
       'DHI_t_mean_wk', 'DHI_t_std_wk', 'DNI_t_min_wk', 'DNI_t_max_wk',
       'DNI_t_mean_wk', 'DNI_t_std_wk', 'GHI_t_min_wk', 'GHI_t_max_wk',
       'GHI_t_mean_wk', 'GHI_t_std_wk', 'PrecipW_t_min_wk', 'PrecipW_t_max_wk',
       'PrecipW_t_mean_wk', 'PrecipW_t_std_wk', 'Humid_t_min_wk',
       'Humid_t_max_wk', 'Humid_t_mean_wk', 'Humid_t_std_wk', 'Press_t_min_wk',
       'Press_t_max_wk', 'Press_t_mean_wk', 'Press_t_std_wk',
       'WindDir_t_min_wk', 'WindDir_t_max_wk', 'WindDir_t_mean_wk',
       'WindDir_t_std_wk', 'WindSpd_t_min_wk', 'WindSpd_t_max_wk',
       'WindSpd_t_mean_wk', 'WindSpd_t_std_wk', 'GHI_t+1',
       'T

In [24]:
# drop columns with NaN introduced becaused of lag and rolling features
sol_temp.dropna(inplace=True)

In [25]:
# re-order features order for convenience
sol_temp = sol_temp[
    [
        'Year_t',
        'Month_t',
        'Year_month_t',
        'Temp_t',
        'Temp_t_min_wk',
        'Temp_t_max_wk',
        'Temp_t_mean_wk',
        'Temp_t_std_wk',
        'Temp_t_mean_wk_velocity',
        'CloudOp_t',
        'CloudOp_t_min_wk',
        'CloudOp_t_max_wk',
        'CloudOp_t_mean_wk',
        'CloudOp_t_std_wk',
        'CloudOp_t_mean_wk_velocity',
        'PrecipW_t',
        'PrecipW_t_min_wk',
        'PrecipW_t_max_wk',
        'PrecipW_t_mean_wk',
        'PrecipW_t_std_wk',
        'PrecipW_t_mean_wk_velocity',
        'Humid_t',
        'Humid_t_min_wk',
        'Humid_t_max_wk',
        'Humid_t_mean_wk',
        'Humid_t_std_wk',
        'Humid_t_mean_wk_velocity',        
        'Press_t',
        'Press_t_min_wk',
        'Press_t_max_wk',
        'Press_t_mean_wk',
        'Press_t_std_wk',
        'Press_t_mean_wk_velocity',
        'WindDir_t',
        'WindDir_t_min_wk',
        'WindDir_t_max_wk',
        'WindDir_t_mean_wk',
        'WindDir_t_mean_wk_velocity',
        'WindDir_t_std_wk',
        'WindSpd_t',
        'WindSpd_t_min_wk',
        'WindSpd_t_max_wk',
        'WindSpd_t_mean_wk',
        'WindSpd_t_std_wk',
        'WindSpd_t_mean_wk_velocity',
        'DNI_t',
        'DNI_t_min_wk',
        'DNI_t_max_wk',
        'DNI_t_mean_wk',
        'DNI_t_std_wk',
        'DNI_t_mean_wk_velocity',
        'DHI_t',
        'DHI_t_min_wk',
        'DHI_t_max_wk',
        'DHI_t_mean_wk',
        'DHI_t_std_wk',
        'DHI_t_mean_wk_velocity',
        'GHI_t',
        'GHI_t_min_wk',
        'GHI_t_max_wk',
        'GHI_t_mean_wk',
        'GHI_t_std_wk',
        'GHI_t_mean_wk_velocity',
        'GHI_t+1'
    ]
]

In [26]:
# check dataframe size and content
sol_temp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1075 entries, 2019-01-07 to 2021-12-16
Data columns (total 64 columns):
 #   Column                      Non-Null Count  Dtype    
---  ------                      --------------  -----    
 0   Year_t                      1075 non-null   int64    
 1   Month_t                     1075 non-null   int64    
 2   Year_month_t                1075 non-null   period[M]
 3   Temp_t                      1075 non-null   float64  
 4   Temp_t_min_wk               1075 non-null   float64  
 5   Temp_t_max_wk               1075 non-null   float64  
 6   Temp_t_mean_wk              1075 non-null   float64  
 7   Temp_t_std_wk               1075 non-null   float64  
 8   Temp_t_mean_wk_velocity     1075 non-null   float64  
 9   CloudOp_t                   1075 non-null   float64  
 10  CloudOp_t_min_wk            1075 non-null   float64  
 11  CloudOp_t_max_wk            1075 non-null   float64  
 12  CloudOp_t_mean_wk           1075 non-null   

In [27]:
# check first entries of the dataframe, look for inconsistent values despite proper formats
sol_temp.head(3)

,Year_t,Month_t,Year_month_t,Temp_t,Temp_t_min_wk,Temp_t_max_wk,Temp_t_mean_wk,Temp_t_std_wk,Temp_t_mean_wk_velocity,CloudOp_t,...,DHI_t_mean_wk,DHI_t_std_wk,DHI_t_mean_wk_velocity,GHI_t,GHI_t_min_wk,GHI_t_max_wk,GHI_t_mean_wk,GHI_t_std_wk,GHI_t_mean_wk_velocity,GHI_t+1
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-07,2019,1,2019-01,5.3875,3.1875,6.8625,4.622421,1.241887,-0.378571,72.0000,...,42.571429,7.780730,1.357143,42.25,32.5,53.25,43.051587,7.883205,1.357143,62.000000
2019-01-08,2019,1,2019-01,5.8125,3.1875,5.8125,4.472421,0.956951,-0.150000,56.0875,...,45.625000,8.937508,3.053571,62.00,32.5,62.00,46.623016,10.049912,3.571429,115.250000
2019-01-09,2019,1,2019-01,3.9750,3.1875,5.8125,4.441865,0.970828,-0.030556,25.5000,...,50.958333,17.446993,5.333333,115.25,32.5,115.25,56.150794,27.917983,9.527778,65.666667


### Save data

In [28]:
# save formated dataset
save_path = '../data/formated/'
save_name = 'sol_temp.csv'
sol_temp.to_csv(save_path + save_name)